In [ ]:
import h2o
import pandas as pd
from h2o.estimators.word2vec import H2OWord2vecEstimator
from nltk.corpus import stopwords
h2o.init()

In [ ]:
#h2o.cluster().shutdown()

In [ ]:
data_path = "/home/mackenzie/workspace/PycharmProjects/DAADRISE_AbusiveLangProject/featureExtraction/EnglishCleanedData.csv"
data = h2o.import_file(data_path)
#print(data)

In [ ]:
def tokenize(sentences):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()),:]
    return tokenized_words

In [ ]:
print("Break tweets into sequence of words")
words = tokenize(data["cleaned_tweet"])

In [ ]:
w2v_frame = h2o.import_file("/home/mackenzie/workspace/PycharmProjects/DAADRISE_AbusiveLangProject/Code_Examples/glove.twitter.27B.200d.txt")
w2v_model = H2OWord2vecEstimator(pre_trained = w2v_frame, vec_size = 200)

In [ ]:
print("Calculate a vector for each tweet")
tweet_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")
print(tweet_vecs)
print(len(tweet_vecs))

In [ ]:
tweets_pd = tweet_vecs.as_data_frame()
print(tweets_pd.isna())

In [ ]:
print(tweets_pd[tweets_pd.isna().any(axis=1)][tweets_pd.columns].head())

In [ ]:
#h2o.download_all_logs()